Passo 1: Importando os modulos necessários

In [ ]:
import pandas as pd
import datetime
import yfinance as yf
from matplotlib import pyplot as plt
import mplcyberpunk
import smtplib
from email.message import EmailMessage

Passo 2: Pegar dados no Yahoo Finance

In [ ]:
# 1. ATUALIZE O ATIVO DO DÓLAR
ativos = ['^BVSP', 'USDBRL=X'] 

hoje = datetime.datetime.now()
um_ano_atras = hoje - datetime.timedelta(days = 365)

# 2. BAIXA OS DADOS
dados_mercado = yf.download(ativos, um_ano_atras, hoje)

# Exibe o topo do DataFrame (para ver a estrutura)
display(dados_mercado.head())

# 3. APLICA A SOLUÇÃO ANTERIOR (que agora vai funcionar)
# Seleciona a coluna 'Adj Close' para todos os ativos, tratando o MultiIndex
dados_fechamento = dados_mercado.loc[:, 'Close'] 

# Renomeia as colunas
dados_fechamento.columns = ['Ibovespa', 'Dólar'] 
# *Note que a ordem segue a ordem dos ativos na lista: ^BVSP, USDBRL=X*

# Remove as linhas com valores ausentes
dados_fechamento = dados_fechamento.dropna()

display(dados_fechamento.head())

Passo 3.1: Manipulando os dados - seleção e exclusão de dados

In [ ]:
# Seu código de download anterior...
# ativos = ['^BVSP', 'USDBRL=X'] 
# dados_mercado = yf.download(ativos, um_ano_atras, hoje) 

# --- NOVO BLOCO DE TRATAMENTO ---

# 1. Tenta selecionar a coluna 'Close' (Fechamento), que é mais universal
# Em um MultiIndex, isso seleciona a coluna 'Close' para ambos os tickers.
# Isso retorna um DataFrame 
dados_fechamento = dados_mercado.loc[:, 'Close'] 

# 2. Renomeia as colunas
# A ordem das colunas no DataFrame (como o Pandas as organiza) é:
# Ticker 1, Ticker 2, ...
# Portanto, a ordem dos nomes deve ser: ^BVSP, USDBRL=X
dados_fechamento.columns = ['Ibovespa', 'Dólar'] 

# 3. Remove as linhas com valores ausentes
dados_fechamento = dados_fechamento.dropna()

# Exiba o resultado final
display(dados_fechamento.head())

Passo 3.2: Manipulando os dados - Criando tabelas com outros timeframes

In [ ]:
dados_fechamento_mensal = dados_fechamento.resample('ME').last()
dados_fechamento_anual = dados_fechamento.resample('YE').last()
dados_fechamento_diario = dados_fechamento.resample('D').last()
dados_fechamento_diario


Passo 4: Calcular fechamento do dia, retorno no ano e retorno no mês dos ativos.

In [ ]:
retorno_no_ano = dados_fechamento_anual.pct_change().dropna()
retorno_no_mes = dados_fechamento_mensal.pct_change().dropna()
retorno_no_dia = dados_fechamento_diario.pct_change().dropna()
retorno_no_dia


Passo 5: Localizar o fechamento do dia anterior, retorno no mês e retorno no ano.

In [ ]:
retorno_no_dia.loc['2025-10-10']

In [ ]:
retorno_no_dia.iloc[4, 0]

In [ ]:
retorno_dia_dolar = retorno_no_dia.iloc[-1, 0]
retorno_dia_ibov = retorno_no_dia.iloc[-1, -1]

retorno_mensal_dolar = retorno_no_mes.iloc[-1, 0]
retorno_mensal_ibov = retorno_no_mes.iloc[-1, -1]

retorno_anual_dolar = retorno_no_ano.iloc[-1, 0]
retorno_anual_ibov = retorno_no_ano.iloc[-1, -1]

retorno_dia_dolar

In [ ]:
retorno_dia_dolar = round(retorno_dia_dolar * 100, 2)
retorno_dia_ibov = round(retorno_dia_ibov * 100, 2)

retorno_mensal_dolar = round(retorno_mensal_dolar * 100, 2)
retorno_mensal_ibov = round(retorno_mensal_ibov * 100, 2)

retorno_anual_dolar = round(retorno_anual_dolar * 100, 2)
retorno_anual_ibov = round(retorno_anual_ibov * 100, 2)

retorno_dia_dolar

Passo 6: Fazer os gŕaficos da performance do último dos ativos

In [ ]:
plt.style.use('cyberpunk')

dados_fechamento.plot(y = 'Ibovespa', use_index = True, marker='o', legend = False)

plt.title('Ibovespa')
plt.xlabel('')
plt.savefig('ibovespa.png', dpi = 300)

plt.show()

In [ ]:
plt.style.use('cyberpunk')

dados_fechamento.plot(y = 'Dólar', use_index = True, marker='o', legend = False)

plt.title('Dólar')
plt.xlabel('')
plt.savefig('dolar.png', dpi = 300)

plt.show()

Passo 7: Enviar e-mail
https://myaccount.google.com/apppasswords

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
email = os.environ.get('EMAIL_USER')
senha = os.environ.get('EMAIL_PASS')

In [ ]:
load_dotenv()

In [ ]:
senha = os.environ.get('senha')
email = 'rsdcruz97@gmail.com'

In [ ]:
msg = EmailMessage()
msg['Subject'] = 'Enviando e-mail com o Python'
msg['From'] = 'rsdcruz97@gmail.com'
msg['To'] = 'raphaelsoaresrv@gmail.com'

msg.set_content(f''' Prezado diretor, segue o relatório diário:

Bolsa: 

No ano o Ibovespa está tendo uma rentabilidade de {retorno_anual_ibov}%, 
enquanto no mês a rentabilidade é de {retorno_mensal_ibov}%

No último dia útil, o fechamento do Ibovespa foi de {retorno_dia_ibov}%.


Dólar:

No ano o Dólar está tendo uma rentabilidade de {retorno_anual_dolar}%,
enquanto no mês a rentabilidade é de {retorno_mensal_dolar}%

No último dia útil, o fechamento do Dólar foi de {retorno_dia_dolar}%.


Abs, 

o melhor estudante de Python da Galaxia

... ''')

In [ ]:
with open('dolar.png', 'rb') as content_file:
    content = content_file.read()
    msg.add_attachment(content, maintype='application', subtype='png', filename='dolar.png')
    
with open('ibovespa.png', 'rb') as content_file:
    content = content_file.read()
    msg.add_attachment(content, maintype='application', subtype='png', filename='dolar.png')

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

email = os.environ.get('EMAIL_USER').strip() # Boa prática!
# --- ADICIONE .strip() NA SENHA ---
senha = os.environ.get('EMAIL_PASS').strip() 

# ... o restante do seu código ...

# Tente o login
with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
    smtp.starttls()
    smtp.login(email, senha)
    
    print("🎉 SUCESSO! O e-mail foi enviado!")

In [ ]:
import smtplib

# Certifique-se de que 'email' e 'senha' foram lidos corretamente
# (seja do .env, seja da leitura que você confirmou)

print("Tentando login na porta 465 (SSL implícito)...")
try:
    # Use SMTP_SSL e a porta 465
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        # Não precisa de smtp.starttls() com SMTP_SSL
        smtp.login(email, senha) 
        
        print("🎉 SUCESSO! O login funcionou na porta 465!")
        
        # O objeto 'msg' precisa estar definido para enviar o email
        smtp.send_message(msg)
        print("E-mail enviado com sucesso!")
        
except smtplib.SMTPAuthenticationError as e:
    print("❌ ERRO FINAL: Falha de Autenticação na 465. O App Password NÃO está sendo aceito.")
    print("Por favor, verifique se a Senha de App foi gerada para o aplicativo 'E-mail'.")
    
except NameError:
    # Caso você não tenha definido a variável 'msg' ainda
    print("Login OK, mas 'msg' não está definido. Próximo passo é criar o EmailMessage.")

except Exception as e:
    print(f"Erro inesperado: {e}")

In [ ]:
# Mude de SMTP_SSL para SMTP e chame o .starttls()
with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
    smtp.starttls() # Inicia a conexão segura
    smtp.login(email, senha)
    smtp.send_message(msg)